In [77]:
import pandas as pd
import numpy as np
import warnings

In [78]:
warnings.filterwarnings("ignore")

In [79]:
data = pd.read_csv("D:/DataIITM/datafile_sourcelocalization.csv")

In [80]:
data.head()

,rec_x,rec_y,rec_z,roll_r,pitch_r,yaw_r,mag_x,mag_y,mag_z,target_x,target_y,target_z
0,-58.0,-58.0,-58.0,-90.0,-90.0,-90.0,7.090135e-08,7.090135e-08,-2.659524e-23,89.720577,-52.311175,3.709327
1,-58.0,-58.0,-58.0,-90.0,-90.0,-45.0,7.090135e-08,5.013483e-08,-5.013483e-08,62.392870,-28.577314,78.041444
2,-58.0,-58.0,-58.0,-90.0,-90.0,0.0,7.090135e-08,-2.659524e-23,-7.090135e-08,26.755383,45.103964,89.720577
3,-58.0,-58.0,-58.0,-90.0,-90.0,45.0,7.090135e-08,-5.013483e-08,-5.013483e-08,16.640779,98.782895,27.659070
4,-58.0,-58.0,-58.0,-90.0,-90.0,90.0,7.090135e-08,-7.090135e-08,1.791233e-23,41.651419,81.499062,-49.224610


In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15625000 entries, 0 to 15624999
Data columns (total 12 columns):
 #   Column    Dtype  
---  ------    -----  
 0   rec_x     float64
 1   rec_y     float64
 2   rec_z     float64
 3   roll_r    float64
 4   pitch_r   float64
 5   yaw_r     float64
 6   mag_x     float64
 7   mag_y     float64
 8   mag_z     float64
 9   target_x  float64
 10  target_y  float64
 11  target_z  float64
dtypes: float64(12)
memory usage: 1.4 GB


In [82]:
# data.drop("Unnamed: 0" , axis =1, inplace=True)

In [83]:
data.head()

,rec_x,rec_y,rec_z,roll_r,pitch_r,yaw_r,mag_x,mag_y,mag_z,target_x,target_y,target_z
0,-58.0,-58.0,-58.0,-90.0,-90.0,-90.0,7.090135e-08,7.090135e-08,-2.659524e-23,89.720577,-52.311175,3.709327
1,-58.0,-58.0,-58.0,-90.0,-90.0,-45.0,7.090135e-08,5.013483e-08,-5.013483e-08,62.392870,-28.577314,78.041444
2,-58.0,-58.0,-58.0,-90.0,-90.0,0.0,7.090135e-08,-2.659524e-23,-7.090135e-08,26.755383,45.103964,89.720577
3,-58.0,-58.0,-58.0,-90.0,-90.0,45.0,7.090135e-08,-5.013483e-08,-5.013483e-08,16.640779,98.782895,27.659070
4,-58.0,-58.0,-58.0,-90.0,-90.0,90.0,7.090135e-08,-7.090135e-08,1.791233e-23,41.651419,81.499062,-49.224610


In [84]:
from sklearn.preprocessing import StandardScaler 
from sklearn.compose import ColumnTransformer 
from sklearn.model_selection import train_test_split

In [124]:
from tensorflow import keras 
from keras.models import Sequential 
from keras.layers import Dense , Dropout , BatchNormalization , Input
from keras.losses import MeanSquaredError
from keras.metrics import R2Score ,RootMeanSquaredError
from keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

### Splitting The Data into Training , Testing , Validating Sets

In [88]:
features = data.drop(columns=["target_x" , "target_y" , "target_z"])
targets = data[["target_x","target_y","target_z"]]

In [89]:
train_features , test_features , train_targets , test_targets = train_test_split(features, targets , train_size=0.7 , random_state=42)

In [90]:
train_features , val_features , train_targets , val_targets = train_test_split(train_features , train_targets , train_size=0.8 , random_state=42)

In [91]:
print("Total Training Points : ",len(train_features))
print("Total validation Points : ",len(val_features))
print("Total Testing Points : ",len(test_features))

Total Training Points :  8750000
Total validation Points :  2187500
Total Testing Points :  4687500


## Scaling of the Data using Transformers

In [92]:
tf = ColumnTransformer(
    [
        ("Scaling" , StandardScaler() , slice(0,10))
    ]
)

In [93]:
train_f = tf.fit_transform(train_features)

In [94]:
val_f = tf.transform(val_features)
test_f = tf.transform(test_features)

## Model Creation 

In [100]:
model = Sequential()
model.add(Input(shape=(9,)))
model.add(Dense(9 , activation= 'linear'))
model.add(Dense(27 , activation= 'selu'))
model.add(Dense(27 , activation= 'linear'))
model.add(Dense(27 , activation= 'selu'))
model.add(Dense(27 , activation= 'linear'))
model.add(Dense(27 , activation= 'selu'))
model.add(Dense(27 , activation= 'linear'))
model.add(Dense(54 , activation= 'linear'))
model.add(Dense(81 , activation= 'selu'))
model.add(Dense(54 , activation= 'linear'))
model.add(Dense(27 , activation= 'selu'))
model.add(Dense(27 , activation= 'linear'))
model.add(Dense(27 , activation= 'selu'))
model.add(Dense(27 , activation= 'linear'))
model.add(Dense(9 , activation= 'selu'))
model.add(Dense(3 , activation='linear'))

In [101]:
model.compile(optimizer = 'adam' , loss = mean_squared_error , metrics=[R2Score()])

In [102]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_56 (Dense)                │ (None, 9)              │            90 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 27)             │           270 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_59 (Dense)                │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 54)             │         1,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 81)             │         4,455 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 54)             │         4,428 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ (None, 27)             │         1,485 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 9)              │           252 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 3)              │            30 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,570 (72.54 KB)

 Trainable params: 18,570 (72.54 KB)

 Non-trainable params: 0 (0.00 B)

In [103]:
plot_model(model ,to_file='model.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


## Training and Validating the Model

In [104]:
history = model.fit(train_f , train_targets.values , epochs=5, validation_data=[val_f , val_targets.values]) 

Epoch 1/5
273438/273438 ━━━━━━━━━━━━━━━━━━━━ 602s 2ms/step - loss: 1031.0935 - r2_score: 0.2968 - val_loss: 994.7126 - val_r2_score: 0.3216
Epoch 2/5
273438/273438 ━━━━━━━━━━━━━━━━━━━━ 823s 3ms/step - loss: 993.5170 - r2_score: 0.3224 - val_loss: 990.6966 - val_r2_score: 0.3243
Epoch 3/5
273438/273438 ━━━━━━━━━━━━━━━━━━━━ 23663s 87ms/step - loss: 991.0778 - r2_score: 0.3244 - val_loss: 990.7969 - val_r2_score: 0.3243
Epoch 4/5
273438/273438 ━━━━━━━━━━━━━━━━━━━━ 2054s 8ms/step - loss: 990.0338 - r2_score: 0.3249 - val_loss: 991.1248 - val_r2_score: 0.3240
Epoch 5/5
273438/273438 ━━━━━━━━━━━━━━━━━━━━ 1858s 7ms/step - loss: 990.1001 - r2_score: 0.3251 - val_loss: 994.4326 - val_r2_score: 0.3218


In [105]:
model.save("TrainedModels/model.h5")

In [118]:
model2 = Sequential()
model2.add(Input(shape=(9,)))
model2.add(Dense(9 , activation= 'linear'))
model2.add(Dense(27 , activation= 'selu'))
model2.add(Dense(100 , activation= 'linear'))
model2.add(Dense(150 , activation= 'selu'))
model2.add(Dense(200 , activation= 'linear'))
model2.add(Dense(300 , activation= 'selu'))
model2.add(Dense(400 , activation= 'linear'))
model2.add(Dense(500 , activation= 'linear'))
model2.add(Dense(240 , activation= 'selu'))
model2.add(Dense(180 , activation= 'linear'))
model2.add(Dense(90 , activation= 'selu'))
model2.add(Dense(60 , activation= 'linear'))
model2.add(Dense(50 , activation= 'selu'))
model2.add(Dense(27 , activation= 'linear'))
model2.add(Dense(5 , activation= 'selu'))
model2.add(Dense(3 , activation='linear'))

In [120]:
model2.compile(optimizer='adam' , loss = mean_squared_error , metrics=[R2Score()])

In [123]:
model2.fit(train_f , train_targets.values , epochs=5 , validation_data=[val_f , val_targets] )

Epoch 1/5
  4222/273438 ━━━━━━━━━━━━━━━━━━━━ 37:13 8ms/step - loss: 1463.5394 - r2_score: 0.0010

KeyboardInterrupt: 

In [122]:
model2.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_96 (Dense)                │ (None, 9)              │            90 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_97 (Dense)                │ (None, 27)             │           270 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_98 (Dense)                │ (None, 100)            │         2,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_99 (Dense)                │ (None, 150)            │        15,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_100 (Dense)               │ (None, 200)            │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_101 (Dense)               │ (None, 300)            │        60,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_102 (Dense)               │ (None, 400)            │       120,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_103 (Dense)               │ (None, 500)            │       200,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_104 (Dense)               │ (None, 240)            │       120,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_105 (Dense)               │ (None, 180)            │        43,380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_106 (Dense)               │ (None, 90)             │        16,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 60)             │         5,460 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_108 (Dense)               │ (None, 50)             │         3,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_109 (Dense)               │ (None, 27)             │         1,377 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_110 (Dense)               │ (None, 5)              │           140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_111 (Dense)               │ (None, 3)              │            18 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,858,997 (7.09 MB)

 Trainable params: 619,665 (2.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,239,332 (4.73 MB)

In [125]:
model3 = Sequential()
model3.add(Input(shape=(9,)))
model3.add(Dense(27, activation='selu'))
model3.add(Dense(27, activation='selu'))
model3.add(Dense(27, activation='selu'))
model3.add(Dense(54, activation='selu'))
model3.add(Dense(81, activation='selu'))
model3.add(Dropout(0.2))  # Adding dropout for regularization
model3.add(Dense(54, activation='selu'))
model3.add(Dense(27, activation='selu'))
model3.add(Dense(9, activation='selu'))
model3.add(Dense(3, activation='linear'))

model3.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=[RootMeanSquaredError()])

model3.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_112 (Dense)               │ (None, 27)             │           270 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_113 (Dense)               │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_114 (Dense)               │ (None, 27)             │           756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_115 (Dense)               │ (None, 54)             │         1,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_116 (Dense)               │ (None, 81)             │         4,455 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 81)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_117 (Dense)               │ (None, 54)             │         4,428 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_118 (Dense)               │ (None, 27)             │         1,485 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 9)              │           252 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_120 (Dense)               │ (None, 3)              │            30 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,944 (54.47 KB)

 Trainable params: 13,944 (54.47 KB)

 Non-trainable params: 0 (0.00 B)

In [126]:
model3.fit(train_f , train_targets.values , epochs=2 , validation_data=[val_f , val_targets.values])

Epoch 1/2
 23325/273438 ━━━━━━━━━━━━━━━━━━━━ 7:51 2ms/step - loss: 1112.1691 - root_mean_squared_error: 33.3334

KeyboardInterrupt: 